In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt

## Load new matrices: 

In [5]:
path_data = '../../data/standardized_data/'
file_log2fc = os.path.join(path_data, 'result_logfc_matrix_2019_12_10.csv')
file_qval = os.path.join(path_data, 'result_qval_matrix_2019_12_10.csv')
file_bin = os.path.join(path_data, 'result_bin_matrix_2019_12_10.csv')

df_log2fc = pd.read_csv(file_log2fc)
df_qval = pd.read_csv(file_qval)
df_bin = pd.read_csv(file_bin)

## Let's compare old Carey results vs. new Carey results: 

### Load older carey datasets: 

In [6]:
file_old = '../../dep/data/Tn_library_DB.xlsx'
df_old = pd.read_excel(file_old)
cols_old = [col for col in df_old.columns if 'Carey' in col]

In [7]:
cols_new = [col for col in df_bin.columns if 'carey' in col]

In [8]:
col_map = list(zip(cols_old, cols_new))
col_map

[('2018_Carey_1A', 'carey_621_vs_mbio_H37Rv'),
 ('2018_Carey_1B', 'carey_630_vs_mbio_H37Rv'),
 ('2018_Carey_1C', 'carey_631_vs_mbio_H37Rv'),
 ('2018_Carey_1D', 'carey_632_vs_mbio_H37Rv'),
 ('2018_Carey_1E', 'carey_641_vs_mbio_H37Rv'),
 ('2018_Carey_1F', 'carey_662_vs_mbio_H37Rv'),
 ('2018_Carey_1G', 'carey_663_vs_mbio_H37Rv'),
 ('2018_Carey_1H', 'carey_667_vs_mbio_H37Rv')]

In [9]:
list_num_both_CES = []
list_num_both_nonCES = []
list_num_CES_old_nonCES_new = []
list_num_nonCES_old_CES_new = []

for cols in col_map:
    
    df_bin_col = df_bin[['Rv_ID', cols[1]]].copy()
    df_old_col = df_old[['Rv_ID', cols[0]]].copy()
    df_bin_both = df_bin_col.merge(df_old_col, how = 'inner', on = 'Rv_ID')

    num_both_CES = df_bin_both[ (df_bin_both[cols[0]]==1) & (df_bin_both[cols[1]]==1) ].shape[0]
    num_both_nonCES = df_bin_both[ (df_bin_both[cols[0]]==0) & (df_bin_both[cols[1]]==0) ].shape[0]
    num_CES_old_nonCES_new = df_bin_both[ (df_bin_both[cols[0]]==1) & (df_bin_both[cols[1]]==0) ].shape[0]
    num_nonCES_old_CES_new = df_bin_both[ (df_bin_both[cols[0]]==0) & (df_bin_both[cols[1]]==1) ].shape[0]

    list_num_both_CES.append(num_both_CES)
    list_num_both_nonCES.append(num_both_nonCES)
    list_num_CES_old_nonCES_new.append(num_CES_old_nonCES_new)
    list_num_nonCES_old_CES_new.append(num_nonCES_old_CES_new)
    
df_map = pd.DataFrame()
df_map['screen'] = [col[1] for col in col_map]
df_map['both_CES'] = list_num_both_CES
df_map['num_both_nonCES'] = list_num_both_nonCES
df_map['num_CES_old_nonCES_new'] = list_num_CES_old_nonCES_new
df_map['num_nonCES_old_CES_new'] = list_num_nonCES_old_CES_new

In [37]:
file_out = '../../dep/data/sanity_check_Carey.csv'
df_map.to_csv(file_out, index = False)

## Let's compare all screens:

In [10]:
file_col_desc = '../../data/column_descriptors_standardized.xlsx'
df_col_desc = pd.read_excel(file_col_desc)

In [11]:
col_map = list(zip(df_col_desc.column_ID.values, df_col_desc.column_ID_2.values))
col_map = [cm for cm in col_map if cm[0] in df_old.columns]

In [51]:
list_num_both_CES = []
list_num_both_nonCES = []
list_num_CES_old_nonCES_new = []
list_num_nonCES_old_CES_new = []

for cols in col_map:
    
    df_bin_col = df_bin[['Rv_ID', cols[1]]].copy()
    df_old_col = df_old[['Rv_ID', cols[0]]].copy()
    df_bin_both = df_bin_col.merge(df_old_col, how = 'inner', on = 'Rv_ID')

    num_both_CES = df_bin_both[ (df_bin_both[cols[0]]==1) & (df_bin_both[cols[1]]==1) ].shape[0]
    num_both_nonCES = df_bin_both[ (df_bin_both[cols[0]]==0) & (df_bin_both[cols[1]]==0) ].shape[0]
    num_CES_old_nonCES_new = df_bin_both[ (df_bin_both[cols[0]]==1) & (df_bin_both[cols[1]]==0) ].shape[0]
    num_nonCES_old_CES_new = df_bin_both[ (df_bin_both[cols[0]]==0) & (df_bin_both[cols[1]]==1) ].shape[0]

    list_num_both_CES.append(num_both_CES)
    list_num_both_nonCES.append(num_both_nonCES)
    list_num_CES_old_nonCES_new.append(num_CES_old_nonCES_new)
    list_num_nonCES_old_CES_new.append(num_nonCES_old_CES_new)
    
df_map = pd.DataFrame()
df_map['screen'] = [col[1] for col in col_map]
df_map['both_CES'] = list_num_both_CES
df_map['both_nonCES'] = list_num_both_nonCES
df_map['CES_old_nonCES_new'] = list_num_CES_old_nonCES_new
df_map['nonCES_old_CES_new'] = list_num_nonCES_old_CES_new

In [64]:
file_out = '../../dep/data/sanity_check_v2.csv'
df_map.to_csv(file_out, index = False)

In [52]:
df_map

,screen,both_CES,both_nonCES,CES_old_nonCES_new,nonCES_old_CES_new
0,griffin_cholesterol_vs_griffin_glycerol,59,3877,37,17
1,griffin_glycerol_vs_mbio_H37Rv,20,3214,647,109
2,zhang_Tyloxapol_pH_6.5_vs_zhang_Tyloxapol_pH_4.5,3,3958,1,28
3,zhang_Tyloxapol_pH_6.5_vs_zhang_pcit_pH_4.5,7,3909,14,60
4,zhang_DETA-NO_pH_7.0_vs_zhang_pH_7.0_no_NO_con...,11,3949,1,29
5,zhang_Trp_Rescue_vs_zhang_in_vitro_control_Rescue,2,3968,10,10
6,korte_2016_otsa_trehalose_vs_korte_2016_otsa_7h9,2,3854,134,0
7,xu_mero_2.5_vs_xu_mero_0,41,3861,69,0
8,carey_621_vs_mbio_H37Rv,8,3759,27,196
9,carey_630_vs_mbio_H37Rv,9,3847,24,110


## Follow-up in more detail on a few:

### Griffing cholesterol:

In [19]:
cols = col_map[0]
df_bin_col = df_bin[['Rv_ID', cols[1]]].copy()
df_old_col = df_old[['Rv_ID', cols[0]]].copy()
df_bin_both = df_bin_col.merge(df_old_col, how = 'inner', on = 'Rv_ID')

In [20]:
df_CES_old_nonCES_new = df_bin_both[ (df_bin_both[cols[0]]==1) & (df_bin_both[cols[1]]==0) ]
df_nonCES_old_CES_new = df_bin_both[ (df_bin_both[cols[0]]==0) & (df_bin_both[cols[1]]==1) ]

Get the gene names:

In [26]:
file_mbio = '../../data/SI_datasets/2017A_DeJesus_Iorger/table_1.xlsx'
df_mbio = pd.read_excel(file_mbio)
df_mbio = df_mbio[['Rv_ID', 'Name']]

In [35]:
df_CES_old_nonCES_new_wNames = df_CES_old_nonCES_new.merge(df_mbio, how = 'inner', on = 'Rv_ID')
df_CES_old_nonCES_new_wNames = df_CES_old_nonCES_new_wNames[['Rv_ID', 'Name', 'griffin_cholesterol_vs_griffin_glycerol', '2011_Griffin_2']]

file_out = '../../dep/data/sanity_check_griffin_CES_OLD_nonCES_NEW.csv'
df_CES_old_nonCES_new_wNames.to_csv(file_out, index = False)

In [36]:
df_nonCES_old_CES_new_wNames = df_nonCES_old_CES_new.merge(df_mbio, how = 'inner', on = 'Rv_ID')
df_nonCES_old_CES_new_wNames = df_nonCES_old_CES_new_wNames[['Rv_ID', 'Name', 'griffin_cholesterol_vs_griffin_glycerol', '2011_Griffin_2']]

file_out = '../../dep/data/sanity_check_griffin_nonCES_OLD_CES_NEW.csv'
df_nonCES_old_CES_new_wNames.to_csv(file_out, index = False)

In [34]:
df_CES_old_nonCES_new_wNames

,Rv_ID,Name,griffin_cholesterol_vs_griffin_glycerol,2011_Griffin_2
0,Rv0009,ppiA,0.0,1
1,Rv0153c,ptbB,0.0,1
2,Rv0202c,mmpL11,0.0,1
3,Rv0655,mkl,0.0,1
4,Rv0695,-,0.0,1
5,Rv0696,-,0.0,1
6,Rv0876c,-,0.0,1
7,Rv1071c,echA9,0.0,1
8,Rv1084,-,0.0,1
9,Rv1096,-,0.0,1


### Korte 2016:

In [59]:
cols = [cm for cm in col_map if 'Korte' in cm[0]][0]
df_bin_col = df_bin[['Rv_ID', cols[1]]].copy()
df_old_col = df_old[['Rv_ID', cols[0]]].copy()
df_bin_both = df_bin_col.merge(df_old_col, how = 'inner', on = 'Rv_ID')

CES in the old dataset, but not the new one: 

In [63]:
df_bin_both[ (df_bin_both[cols[0]]==1) & (df_bin_both[cols[1]]==0) ].Rv_ID.values

array(['Rv0012', 'Rv0092', 'Rv0166', 'Rv0169', 'Rv0191', 'Rv0210',
       'Rv0234c', 'Rv0244c', 'Rv0263c', 'Rv0270', 'Rv0279c', 'Rv0280',
       'Rv0281', 'Rv0386', 'Rv0443', 'Rv0449c', 'Rv0483', 'Rv0492c',
       'Rv0501', 'Rv0570', 'Rv0574c', 'Rv0613c', 'Rv0630c', 'Rv0727c',
       'Rv0746', 'Rv0747', 'Rv0754', 'Rv0767c', 'Rv0782', 'Rv0861c',
       'Rv0877', 'Rv0889c', 'Rv0890c', 'Rv0914c', 'Rv0976c', 'Rv1180',
       'Rv1181', 'Rv1206', 'Rv1212c', 'Rv1232c', 'Rv1235', 'Rv1236',
       'Rv1238', 'Rv1244', 'Rv1262c', 'Rv1266c', 'Rv1283c', 'Rv1323',
       'Rv1442', 'Rv1632c', 'Rv1710', 'Rv1727', 'Rv1737c', 'Rv1745c',
       'Rv1768', 'Rv1770', 'Rv1795', 'Rv1796', 'Rv1820', 'Rv1823',
       'Rv1836c', 'Rv1840c', 'Rv1864c', 'Rv1902c', 'Rv1908c', 'Rv1910c',
       'Rv1971', 'Rv1991c', 'Rv2000', 'Rv2052c', 'Rv2065', 'Rv2066',
       'Rv2074', 'Rv2115c', 'Rv2159c', 'Rv2209', 'Rv2214c', 'Rv2224c',
       'Rv2236c', 'Rv2241', 'Rv2328', 'Rv2329c', 'Rv2394', 'Rv2458',
       'Rv2487c', 'Rv255